# Select the LCLS-II py3 kernel in the top right
# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psana as ps

# Specify experiment and run number. Then generate datasource

In [ ]:
exp = 'tmolv2918'
run_number = 215
ds = ps.DataSource(exp=exp, run=run_number)
run = next(ds.runs())

# Setup smd

In [ ]:
smd = ds.smalldata(filename='test.h5')

In [ ]:
evt0=next(run.events())

In [ ]:
help(smd)

In [ ]:
testData = {'det':np.arange(10)*np.nan, 'epics_det1':0,'epics_det2':np.nan}

In [ ]:
smd.event(evt0, **testData)

In [ ]:
Nbad = 1
Nfound = 1
rebin_factor = 10
mpks = np.array([1,2,3,4])
if smd.summary:
    Nbad = smd.sum(Nbad)
    Nfound = smd.sum(Nfound)
    smd.save_summary(summary_Nfound=Nfound, summary_Nbad=Nbad, summary_rebin_factor=rebin_factor, summary_mpks=mpks.copy())
    
smd.done()

print(smd.summary)

# Open H5 and reorganize H5

In [ ]:
import h5py

In [ ]:
raw = h5py.File("test.h5",'r')

In [ ]:
def generateDataset(raw, key, keyword, out):
    if keyword in key:
        detname = key.replace(keyword,'')
        subfolder = keyword.replace('_','')
        if subfolder not in out:
            out.create_group(subfolder)
        out.create_dataset(name = '/%s/%s' % (subfolder,detname), data = raw[ key ])
        return True
    else:
        return False
    
def reformatH5( origFile, outFile ):
    raw = h5py(origFile, 'r')
    with h5py.File(outFile,'w') as out:
        for key in raw.keys():
            keyword = 'epics_'
            placed = generateDataset(raw, key, keyword, out)
            keyword = 'summary_'
            placed = placed | generateDataset(raw, key, keyword, out)
            if not placed:
                out.create_dataset(name = key, data=raw[key])
        
        